In [2]:
%matplotlib inline

In [63]:
from keras_utilities import *
from keras_utilities.models.vgg16 import Vgg16
from keras_utilities.models.vgg16bn import Vgg16BN
from keras.models import Sequential
from keras.layers import Dense, Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import bcolz
import os
import numpy as np
import pandas as pd

In [74]:
# Constants
PROD = True

if not PROD:
    dataset = 'sample'
else:
    dataset = 'train'

# Moving Data Around

In [5]:
%cd ~/Fisheries/

/home/ubuntu/Fisheries


In [6]:
%ls

analysis.ipynb                  test_stg1/     train.zip
sample/                         test_stg1.zip  val/
sample_submission_stg1.csv.zip  test_stg2.7z
sample_submission_stg2.csv.zip  train/


In [7]:
%ls train/

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [8]:
# ! rm -R /home/ubuntu/Fisheries/sample/

In [9]:
# create_data_sample('train', 'sample')

In [10]:
%ls train/

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [11]:
%ls sample/

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [12]:
! ls -l train/ALB/ | wc -l

1372


In [13]:
! ls -l sample/ALB | wc -l 

427


We're good on counts, and seem to have successfully partitioned out 25% of the data to represent a training sample. Next, we have to work on coming up with a good validation set. Do the training set and validation sets come from different boats? If so, we might want to think about splitting out a few of the training boats to serve as validation boats.

Maybe a new way to handle validation set creation is to allow two types of partitioning:

1. A random percentage of the training data
2. Some subset of the training data that meets a certain criteria (we could start with something simply based on the file's name)

For cases where individual samples are totally iid we can use method 1, whereas if there is some relationship between examples (like some fish coming from the same boat, in this case) we can use method 2.

In [14]:
# TODO: a function for recursively creating a validation set from a training directory
# for subdir in os.listdir('train'):
#     train_path = 'train/' + subdir + '/'
#     val_path = 'val' + '/' + subdir + '/'
#     move_data_subset(train_path, val_path, subset_pct=.2, method='move')

In [15]:
! ls -l val/ALB | wc -l

348


In [16]:
! ls -l train/ALB | wc -l

1372


# Extracting Data

In [81]:
gen = ImageDataGenerator()

In [82]:
train_gen = gen.flow_from_directory(dataset, 
                                    target_size = (224,224), 
                                    batch_size = 32, 
                                    class_mode = 'categorical')

Found 2997 images belonging to 8 classes.


In [83]:
val_gen = gen.flow_from_directory('val',
                                  target_size = (224,224),
                                  batch_size = 32,
                                  class_mode = 'categorical')

Found 784 images belonging to 8 classes.


In [84]:
mdl = Vgg16()

In [85]:
mdl.model.pop()
for layer in mdl.model.layers: layer.trainable = False
mdl.model.add(Dense(8, activation='softmax'))
mdl.compile()

In [50]:
mdl.model.fit_generator(train_gen, 
                        samples_per_epoch = train_gen.nb_sample, 
                        nb_epoch = 1, 
                        validation_data = val_gen, 
                        nb_val_samples=val_gen.nb_sample)

Epoch 1/1
960/960 [==============================] - 965s - loss: 3.8568 - acc: 0.5437 - val_loss: 2.7484 - val_acc: 0.6480


# Pre-Computing Convolutional Layers

This section is computationally intensive, so everything is commented out.

In [94]:
# mdl = Vgg16BN()

In [95]:
# layer_types = map(lambda layer: type(layer), mdl.model.layers)

In [96]:
# first_dense_index = layer_types.index(Dense)

In [97]:
# new_layers = mdl.model.layers[:first_dense_index]

In [98]:
# conv_out_mdl = Sequential(new_layers)

In [99]:
# conv_out_data = conv_out_mdl.predict_generator(train_gen, train_gen.nb_sample)

In [100]:
# conv_out_data.shape

In [101]:
# bcolz.carray(conv_out_data, rootdir='data/train')

In [102]:
# val_conv_out_data = conv_out_mdl.predict_generator(val_gen, val_gen.nb_sample)

In [103]:
# bcolz.carray(val_conv_out_data, rootdir='data/val')

In [73]:
# ! mv data/train data/sample

In [106]:
conv_out_unflattened_layers = conv_out_mdl.layers[:-1]

In [107]:
conv_out_unflattened_layers

In [108]:
conv_out_unflattened = Sequential(conv_out_unflattened_layers)

In [109]:
conv_out_unflattened_trn = conv_out_unflattened.predict_generator(train_gen, train_gen.nb_sample)

In [112]:
conv_out_unflattened_trn.shape

(2997, 512, 14, 14)

In [113]:
bcolz.carray(conv_out_unflattened_trn, rootdir='data/train_unflattened')

carray((2997, 512, 14, 14), float32)
  nbytes := 1.12 GB; cbytes := 247.62 MB; ratio: 4.63
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 5; chunksize: 2007040; blocksize: 32768
  rootdir := 'data/train_unflattened'
  mode    := 'a'
[[[[  0.           0.           0.         ...,  10.98112297  17.46021652
     10.90853405]
   [  0.           0.           0.         ...,   7.96724606  17.38532066
      5.55078697]
   [  0.           0.           0.         ...,   0.           2.72810388
      0.        ]
   ..., 
   [  0.           0.          35.58968735 ...,   0.           0.           0.        ]
   [  0.           0.          31.45189857 ...,   0.           0.
      1.54978704]
   [  0.           0.           2.15396333 ...,   0.           0.
      7.28270435]]

  [[  0.           0.           0.         ...,   0.           0.           0.        ]
   [  0.           0.           0.         ...,   0.           0.           0.        ]
   [  0.      

In [114]:
conv_out_unflattened_val = conv_out_unflattened.predict_generator(val_gen, val_gen.nb_sample)

In [115]:
conv_out_unflattened_val.shape

(784, 512, 14, 14)

In [116]:
bcolz.carray(conv_out_unflattened_val, rootdir='data/val_unflattened')

carray((784, 512, 14, 14), float32)
  nbytes := 300.12 MB; cbytes := 65.10 MB; ratio: 4.61
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 2; chunksize: 802816; blocksize: 32768
  rootdir := 'data/val_unflattened'
  mode    := 'a'
[[[[  0.           0.           0.         ...,   0.           0.           0.        ]
   [  0.           0.           0.         ...,   0.           0.           0.        ]
   [  0.           0.           0.         ...,   0.           0.           0.        ]
   ..., 
   [ 16.43975639  52.57017517  63.05947113 ...,   0.           0.           0.        ]
   [ 13.01502037  39.43035507  50.86906052 ...,   0.           0.           0.        ]
   [  3.04450369  10.44467449  18.17323112 ...,   0.           0.           0.        ]]

  [[  8.06585217   0.           0.         ...,   0.           0.           0.        ]
   [  0.           0.           0.         ...,   0.           0.           0.        ]
   [  0.           0.

In [117]:
! ls data

sample	train  train_unflattened  val  val_unflattened
